# 1. LLM

In [1]:
import torch
from transformers import BitsAndBytesConfig
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.llms.huggingface_pipeline import HuggingFacePipeline

model_name: str = "microsoft/phi-2"

nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=nf4_config,
    low_cpu_mem_usage=True
)


tokenizer = AutoTokenizer.from_pretrained(model_name)
max_new_token = 256

model_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=max_new_token,
    pad_token_id=tokenizer.eos_token_id
)

gen_kwargs = {
    "temperature": 1
}

llm = HuggingFacePipeline(
    pipeline=model_pipeline,
    model_kwargs=gen_kwargs
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# 2. Prompt template

In [2]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(""""Instruct:{prompt}\nOutput:"                                 
"""
)

# 3. Chain

In [3]:
chain = prompt | llm

In [4]:
output = chain.invoke(
    {
        "prompt": "Write a detailed analogy between mathematics and a lighthouse."
    }
)

In [5]:
print(output)

"Instruct:Write a detailed analogy between mathematics and a lighthouse.
Output:"                                 

# Solution

# Analogy between Mathematics and a Lighthouse

# Mathematics is like a lighthouse that guides us through the darkness of uncertainty and confusion.
# It provides us with a clear and precise way of understanding and solving problems.
# It helps us to navigate through the complex and dynamic world of numbers, shapes, patterns, and logic.
# It enables us to discover new insights and possibilities that would otherwise be hidden or inaccessible.
# It empowers us to communicate and collaborate with others who share our passion and curiosity for knowledge.
# It inspires us to explore and experiment with different methods and tools to enhance our creativity and innovation.
# It challenges us to think critically and rigorously about the validity and reliability of our claims and arguments.
# It rewards us with a sense of satisfaction and accomplishment when we master 